## 02. Upstage OCR 시작/중지 ( API / Batch )
### 1. Subscribe to the model package

In [5]:

import boto3
import sagemaker
from sagemaker import ModelPackage
# from sagemaker import ModelPackage, get_execution_role
import json

In [13]:
sagemaker_session = sagemaker.Session()
sagemaker_runtime = boto3.client("sagemaker-runtime")
s3 = boto3.client('s3')

# role = get_execution_role()
role = "AmazonSageMaker-ExecutionRole-20240820T145192"
s3_bucket = "gsshop-video-analysis-761482380245-ap-northeast-2"
key = "images/prd_01/vrid_00001/_SUCCUESS"
prefix = key.replace("/_SUCCUESS","")

endpoint_config_name = "Upstage-Document-OCR-config"



In [3]:
model_package_name = "upstage-document-ocr-240704-r1-8b4651227fa23fc6925dedbc4b4d1437"

# Mapping for Model Packages
model_package_map = {
    "us-east-1": f"arn:aws:sagemaker:us-east-1:865070037744:model-package/{model_package_name}",
    "us-east-2": f"arn:aws:sagemaker:us-east-2:057799348421:model-package/{model_package_name}",
    "us-west-1": f"arn:aws:sagemaker:us-west-1:382657785993:model-package/{model_package_name}",
    "us-west-2": f"arn:aws:sagemaker:us-west-2:594846645681:model-package/{model_package_name}",
    "ca-central-1": f"arn:aws:sagemaker:ca-central-1:470592106596:model-package/{model_package_name}",
    "eu-central-1": f"arn:aws:sagemaker:eu-central-1:446921602837:model-package/{model_package_name}",
    "eu-west-1": f"arn:aws:sagemaker:eu-west-1:985815980388:model-package/{model_package_name}",
    "eu-west-2": f"arn:aws:sagemaker:eu-west-2:856760150666:model-package/{model_package_name}",
    "eu-west-3": f"arn:aws:sagemaker:eu-west-3:843114510376:model-package/{model_package_name}",
    "eu-north-1": f"arn:aws:sagemaker:eu-north-1:136758871317:model-package/{model_package_name}",
    "ap-southeast-1": f"arn:aws:sagemaker:ap-southeast-1:192199979996:model-package/{model_package_name}",
    "ap-southeast-2": f"arn:aws:sagemaker:ap-southeast-2:666831318237:model-package/{model_package_name}",
    "ap-northeast-2": f"arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/{model_package_name}",
    "ap-northeast-1": f"arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/{model_package_name}",
    "ap-south-1": f"arn:aws:sagemaker:ap-south-1:077584701553:model-package/{model_package_name}",
    "sa-east-1": f"arn:aws:sagemaker:sa-east-1:270155090741:model-package/{model_package_name}",
}

region = sagemaker_session.boto_region_name
if region not in model_package_map.keys():
    raise Exception(f"Current boto3 session region {region} is not supported.")

model_package_arn = (
    model_package_map[region]
)

print(f"Model Package: '{model_package_arn}'")

Model Package: 'arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/upstage-document-ocr-240704-r1-8b4651227fa23fc6925dedbc4b4d1437'


### 2. Create an endpoint and perform real-time inference


In [4]:
model_name = "Upstage-Document-OCR"

real_time_inference_instance_type = (
    "ml.g5.2xlarge"
)

In [6]:
# 2-1. 모델 생성 (최초 1회)
model = ModelPackage(
    role=role,
    name=model_name,
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session
)

model.create(instance_type=real_time_inference_instance_type)

In [7]:
# 2-2. 모델 Endpoint Config 생성 (최초 1회 만 생성)
endpoint_config_name = f"{model_name}-config"
print(f"Endpoint config name: '{endpoint_config_name}'")

endpoint_config = sagemaker_session.create_endpoint_config(
    name=endpoint_config_name,
    model_name=model.name,
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type
)


Endpoint config name: 'Upstage-Document-OCR-config'


In [10]:
import re

def sanitize_endpoint_name(name):
    # 알파벳, 숫자, 하이픈만 허용하고 나머지는 제거
    sanitized = re.sub(r'[^a-zA-Z0-9-]', '', name)
    
    # 이름이 하이픈으로 시작하거나 끝나면 제거
    sanitized = sanitized.strip('-')
    
    # 이름이 50자를 초과하면 자르기
    if len(sanitized) > 50:
        sanitized = sanitized[:50]
    
    # 이름이 비어있거나 숫자로 시작하면 접두사 추가
    # if not sanitized or sanitized[0].isdigit():
        # sanitized = 'endpoint-' + sanitized
    
    sanitized = 'endpoint-' + sanitized
    
    return sanitized

# 2-3. Endpoint 생성 및 배포
endpoint_name = sanitize_endpoint_name(prefix)


In [11]:
# 2-3. 모델 Endpoint 생성 ( vrid 이용하여 유일하게 생성)

print(f"Creating endpoint: '{endpoint_name}'")
endpoint = sagemaker_session.create_endpoint(
    endpoint_name=endpoint_name,
    config_name=endpoint_config_name,
    wait=True
)

print(f"Created endpoint: '{endpoint_name}'")


Creating endpoint: 'imagesprd01vrid00001-endpoint'
---------!Created endpoint: 'imagesprd01vrid00001-endpoint'


In [14]:
### 2-4. s3에서 image 목록 불러오기
def list_s3_keys(bucket_name, prefix):
    
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    
    keys = []
    if 'Contents' in response:
        keys = [obj['Key'] for obj in response['Contents']]
    
    return keys

### 2-5. s3에서 image 다운로드
def download_from_s3(bucket, key):
    """S3에서 파일을 다운로드하고 바이너리 데이터와 Content-Type을 반환합니다."""
    response = s3.get_object(Bucket=bucket, Key=key)
    content_type = response['ContentType']
    data = response['Body'].read()
    return data, content_type

### 2-6. Upstage OCR Endpoint 호출
def invoke_endpoint(endpoint_name, image_data, content_type):
    """SageMaker 엔드포인트를 호출합니다."""
    response = sagemaker_runtime.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType=content_type,
        Body=image_data
    )
    return json.loads(response["Body"].read().decode('utf-8'))

# S3 버킷 이름과 경로 설정
# s3_bucket = "gsshop-video-analysis-com-761482380245"
# prefix = "images/prd_01/vrid_00001/"

# 키 목록 조회
keys = list_s3_keys(s3_bucket, prefix)
input_ocr_list = "<input_ocr_list>\n"

# 결과 출력
print(f"Total keys found: {len(keys)}")
i = 0
for s3_key in keys:
    if prefix != s3_key and f"{prefix}_SUCCESS" != s3_key:
        image_data, content_type = download_from_s3(s3_bucket, s3_key)
        
        # 엔드포인트 호출 (S3에서 가져온 Content-Type 사용)
        response = invoke_endpoint(endpoint_name, image_data, content_type)
        
        # print("Endpoint response:")
        # print(response["text"])
        # print(json.dumps(response, indent=2))
        
        original_ocr_text = response["text"]
        
        ## OCR만 적용
        input_ocr_list += "<frame>\n"
        input_ocr_list += "  <frame_id>{}</frame_id>\n".format(i)
        input_ocr_list += "  <original_ocr_text>{}</original_ocr_text>\n".format(original_ocr_text)
        # input_ocr_list += "  <croped_ocr_text>{}</croped_ocr_text>\n".format(croped_ocr_text)
        input_ocr_list += "</frame>\n"
        
        i = i+1
input_ocr_list += "</input_ocr_list>"

print(input_ocr_list)
print(f"Upstage Call Ended")


Total keys found: 62
<input_ocr_list>
<frame>
  <frame_id>0</frame_id>
  <original_ocr_text>healthyhug 1개월 2개회 SOZ 소연금 로이진1200 펀드로이천1200 알파플러스 커피플러스 HACCP h 소연골콘드로이친복합물 (소인골주출분말 40%) 66.7% 소연골 콘드로이친 1200 소연골란드로미인복합물 1200mg(0정) 900mg x60정 (54g/200kcal) healthyhug healthyfu 연결 트마전1200 드로어진1200 선택1 12박스 - 리스 + 의미플러스 279,000 무12 소연골큰드로이친복합물 소연골몬드로이친복합물 앱할인 5만원시 (소연골추출분말 40%) 66.7% (소연골추출분말 40%) 66.7% healthyf 아래 12:00 무엇릿 1200 추언중반드로이전복밥을 1200mg(2일) 소인을랜드로이전복합물 1200mg/ 소연금) YERT 비스 월 19,084원 글로벌 900mg N 60점 (54g/200kcal) 900mgx60정 (54g/ 콘드트 열매를 healthiphu 선택2 6박스 소연골콘드로이친복합물 소인물 (소연골주출분말 40%) 66.7% 199,000 무6 코드로바람(10 콘드레이진복잡을 1000mg(7월) 0mg x60점 (54g/200kcal) 앱할인 2만원시 알파들리  소연골콘트로이친복합물 월 29,834원 (소연골 추출분말 40%) 66.7% 6인골반드로이전백점을 1200mg/3점 900mg · 60점 (54g/200kg *1일 2회, 1회 1정 섭취 시 1박스 1개월분 1일 2회, 1회 1정 섭취 시 1박스 1개월분 소연골추출분말/인도 자동 080-469-4545</original_ocr_text>
</frame>
<frame>
  <frame_id>1</frame_id>
  <original_ocr_text>healthyhug 건 강을 품 다 [ 소연골 콘드로이친 1200 선택2 6박스 199,000무6 월 33,167원 12

In [19]:
### 2.7 프롬프트 정의

system_prompt = """<instruction>
- 당신은 홈쇼핑 방송에서 정확한 가격정보를 추출하는 머신입니다.
- 답변결과는 다음과 <result> 태그 예시와 같이 json형태로 출력해주세요.
- json 결과 앞뒤에 부가 설명없이 반드시 json 결과로 알려주세요.
- 최종 결과 값이 json 형태가 맞는지 반드시 다시 확인 해주세요.
</instruction>

<result>
{
    "대표상품명": "[대표상품명]",
    "가격정보":
    [
        {
            "선택명": "[선택명]",
            "판매가격": "[판매가격]",
            "무이자개월수": "[무이자개월수]",
            "앱할인가": "[앱할인가]",
            "일시불할인가" "[일시불할인가]"
        },
        {
            "선택명": "[선택명]",
            "판매가격": "[판매가격]",
            "무이자개월수": [무이자개월수],
            "앱할인가": "[앱할인가]",
            "일시불할인가" "[일시불할인가]"
        }
    ],
    "행사카드할인율": "[행사카드할인율]",
    "가격행사": ["[특가방송]", "[세일방송]"] 

}
</result>
"""

# - <croped_ocr_text> 는 Frame에서 우측 바를 Crop한 이미지에 대한 OCR입니다. 
# - <croped_ocr_text> 에 가격정보가 정확합니다. <croped_ocr_text> 를 먼저 참조하고, <croped_ocr_text> 가격정보가 없다면 <original_ocr_text>를 부가적으로 참조해주세요.

final_prompt = """

<instruction>
- <input_ocr_list> 는 홈쇼핑 동영상내 여러 frame으로부터 ocr text 목록이고, Hallucinations이 있을 수 있습니다.
- <input_ocr_list> 으로부터 대표상품명/가격/행사카드할인율/가격행사 정보를 추출해주세요.
- <original_ocr_text> 는 전체 Frame에서 OCR로 문자를 추출하였습니다.
</instruction>

<input_ocr_list/>

<가격정보_유의사항>
 - 가격은 선택옵션으로 2개 이상이 될 수 있습니다. 2개 이상일 경우 "선택1", "선택2" 이렇게 표시가 있는 경우가 많으나 없을수도 있습니다. 
 - 선택사항이 있을수 있으니, 가격이 한개라도 복수개로 표현해주세요. ex) json 표현으로 [] 
 - 가격이 나열되는 순서는 [대표상품명][선택명][판매가격][무이자개월수][앱할인가][일시불할인가] 순으로 많이 나옵니다.
 - 예를 들어 "아마존 영양제 선택1 12박스 279,000 무12 앱할인 5만원시 월 19,084원 선택2 6박스 199,000 무6 앱할인 2만원시 월 29,834원 일시불 7만원 " 이런 OCR은 다음과 같이 정보가 Mapping 됩니다.
    1. 대표상품명 : "아마존 영양제"
    2. 가격
        2.1 선택명 : "선택1 12박스"
        2.2 판매가격 : "279,000"
        2.3 무이자개월수 : "무12"
        2.4 앱할인가 : "앱할인 5만원"
        2.5 일시불할인가 : "일시불 7만원"
</가격정보_유의사항>

<선택명유의사항>
- 선택명은 구매옵션을 구분하는 기준입니다. 절대 사이즈(S/M/L/XL)나 색상을 구분하는 내용은 선택명이 아닙니다.
- 구매옵션이 1개라면 단일상품입니다. 선택명이 없습니다. "선택[숫자]" 이런 형태의 글자가 없다면 선택명은 "없음"으로 표시해주세요.
- '선택[숫자]', '선택[숫자]' 형식인 경우, [숫자]가 작은 순서로 먼저 나와야 합니다.
- '1종', '2종', '1박스', '2박스' 등 '선택'이라는 Text가 없는 형식인 경우, '판매가격'이 비싼 순서로 나열하세요.
</선택명유의사항>

- 추출해야 될 속성명은 다음과 같습니다.
  1. 대표상품명 : 해당상품명을 추출해주세요. OCR 가장 앞이나 가격정보 앞쪽에 대표상품명이 위치합니다.
  2. 가격 : <가격정보_유의사항> 을 참조하여 2.1 선택명부터 2.4판매가격까지 정보를 추출해주세요.
    2.1. 선택명 : <선택명유의사항> 유의하여 선택명을 작성해주세요.
    2.2. 판매가격 : 숫자 + 통화로 표현해주세요. 위 <가격유의사항>을 유의하여 앱할인,행사카드할인 적용이 되지 않은 원가격을 표시해주세요. ex) xx,xxx 원
    2.3. 무이자개월수 : 할부시 무이자가능한 개월수를 '무x' 형식으로 표기합니다. 예를 들어 '무12'는 무이자 12개월을 뜻합니다. 무이자개월수는 가격 정보와 붙어있을 수 있습니다. 예를 들어 '36,900무6'은 가격이 '36,900원'이고 무이자개월수는 '6개월' 입니다. 반드시 '무x' 형식으로 알려주세요. ex) 무3, 무6 or 무12. 무이자개월수는 가격 뒤에 또는 별도로 표기될 수 있습니다. 예를 들어, '89,000 무3' 또는 '89,000원 무3' 또는 '39,000무3' 형식으로 표시됩니다. 가격과 무이자개월수 사이에 공백이 있을 수도, 없을 수도 있습니다.
    2.4. 앱할인가 : 앱으로 구매시 추가 할인 가격 or 추가 할인율을 말합니다. - '앱할인 5만원시' or '앱주문 5만원 할인' or '앱할인율 10 %' 이런 문구가 있으면 '앱할인 5만원' or '앱할인 10%' 라고 알려주세요. ex) 앱할인 x만원, ex) 앱할인 x%
  3. 행사카드할인율 : 해당 제품의 특별 행사기간에 특정 카드 이용시 추가 할인이 가능한 정보입니다. '행사카드 5% 할인시' 이렇게 표현이 되어 있다면 '행사카드 5% 할인' 라고 알려주세요. ex) '행사카드 x% 할인'
  4. 가격행사 : '특가' 또는 '세일' 문구가 있다면 가격행사 대상 방송입니다. '세일'은 영어인 sale, SALE로 표기될 수 있습니다. 이런 문구가 있다면 '특가방송', '세일방송'이라고 알려주세요. 결과값에는 "가격행사" 항목으로 "특가방송" 또는 "세일방송"을 포함한 리스트로 표시해주세요. ex1) '가격행사: ["특가방송"]', ex2)'가격행사: ["세일방송"]', ex3)'가격행사: ["특가방송", "세일방송"]'

- 추출할 수 있는 값이 없다면 "없음"으로 표현해주세요.

- 다음은 추출 되어진 예시입니다.
<example-01>
    <input_ocr_list>
        <frame>
                <frame_id>0</frame_id>
            <ocr>
                아마존 영양제 선택1 12박스 279,000 무12 앱할인 5만원시 월 19,084원 선택2 6박스 199,000 무6 앱할인 2만원시 월 29,834원 앱주문 2만원 할인 행사카드5% 할인 시 170,050무6 월 28,342원
            </ocr>
        </frame>
    </input_ocr_list>
    <result>
    {
        "대표상품명": "아마존 영양제",
        "가격":
        [
            {
                "선택명": "선택1 12박스",
                "판매가격": "279,000원",
                "무이자개월수": 무12,
                "앱할인가": "앱할인 5만원",
                "일시불할인가" "없음"

            },
            {
                "선택명": "선택2 6박스", 
                "가격": "199,000원",
                "무이자개월수": 무6,
                "앱할인가": "앱할인 2만원",
                "일시불할인가" "없음"
            }
        ],
        "행사카드할인율": "행사카드 5% 할인",
        "가격행사": "없음"
    }
    </result>
</example-01>

<example-02>
    <input_ocr_list>
        <frame>
                <frame_id>0</frame_id>
            <ocr>
                아마존 TV 55형 1,580,000무36 월 43,889원 KB국민, 롯데,삼성,신한, 하나,현대 무 36개월/ 그외 카드 무12개월 2024년 3월 출시 최신상
            </ocr>
        </frame>
        <frame>
                <frame_id>1</frame_id>
            <ocr>
                아마존 TV 65형 2,030,000 무36 월 56,389원 KB국민, 롯데, 삼성, 신한, 하나, 현대 무 36개월/ 그외 카드 무 12개월 TV 구매 전고객 GS가 26만9천원 LG 우퍼사운드바 포함 SP2 본품 설치후 2주내 별도배송
            </ocr>
        </frame>
        <frame>
                <frame_id>2</frame_id>
            <ocr>
                아마존 TV 앱 1만원+ 일시불 3만원 행사 카드 5% 할인 시 [55형] 1,463,000원 [65형] 1,890,500원 1인 1일 한도 20만원 특가
            </ocr>
        </frame>
        <frame>
                <frame_id>3</frame_id>
            <ocr>
                아마존 TV 55형 + TV대 1,680,000무36 월 46,666원 KB국민, 롯데,삼성,신한, 하나,현대 무 36개월/ 그외 카드 무12개월 2024년 3월 출시 최신상
            </ocr>
        </frame>
        <frame>
                <frame_id>4</frame_id>
            <ocr>
                아마존 TV 65형 + TV대 2,130,000 무36 월 59,166원 KB국민, 롯데, 삼성, 신한, 하나, 현대 무 36개월/ 그외 카드 무 12개월 TV 구매 전고객 GS가 26만9천원 LG 우퍼사운드바 포함 SP2 본품 설치후 2주내 별도배송
            </ocr>
        </frame>
        <frame>
                <frame_id>5</frame_id>
            <ocr>
                아마존 TV + TV대 앱 1만원+ 일시불 5만원 행사 카드 5% 할인 시 [55형 + TV대] 1,543,000원 [65형 + TV대] 1,570,500원 1인 1일 한도 20만원 특가
            </ocr>
        </frame>
    </input_ocr_list>
    <result>
    {
        "대표상품명": "아마존 TV",
        "가격":
        [
            {
                "선택명": "65형 + TV대", 
                "판매가격": "2,130,000원",
                "무이자개월수": "무36",
                "앱할인가": "앱할인 1만원",
                "일시불할인가": "일시불 5만원"
            },
            {
                "선택명": "65형", 
                "판매가격": "2,030,000원",
                "무이자개월수": "무36",
                "앱할인가": "앱할인 1만원",
                "일시불할인가": "일시불 3만원"
            },
            {
                "선택명": "55형 + TV대",
                "판매가격": "1,680,000원",
                "무이자개월수": "무36",
                "앱할인가": "앱할인 1만원",
                "일시불할인가": "일시불 5만원"
            },
            {
                "선택명": "55형",
                "판매가격": "1,580,000원",
                "무이자개월수": "무36",
                "앱할인가": "앱할인 1만원",
                "일시불할인가": "일시불 3만원"
            }
        ],
        "행사카드할인율": "행사카드 5% 할인",
        "가격행사": ["특가행사"]
    }
    </result>
</example-02>

- 최종 결과값의 Template는 다음과 같습니다. 반드시 json 형태로 결과를 출력해주세요.
<result>
{
    "대표상품명": "[대표상품명]",
    "가격정보":
    [
        {
            "선택명": "[선택명]",
            "판매가격": "[판매가격]",
            "무이자개월수": "[무이자개월수]",
            "앱할인가": "[앱할인가]",
            "일시불할인가" "[일시불할인가]"
        },
        {
            "선택명": "[선택명]",
            "판매가격": "[판매가격]",
            "무이자개월수": [무이자개월수],
            "앱할인가": "[앱할인가]",
            "일시불할인가" "[일시불할인가]"
        }
    ],
    "행사카드할인율": "[행사카드할인율]",
    "가격행사": ["[특가방송]", "[세일방송]"] 
}
</result>
"""

In [14]:

bedrock_rt = boto3.client(
    "bedrock-runtime",
    region_name="us-west-2",
    # endpoint_url=None                      
)
modelId = "anthropic.claude-3-5-sonnet-20240620-v1:0"

### 2-7 Bedrock Call
def get_final_result(input_ocr_list, system_prompt, final_prompt):
    # start_time = time.time()

    # st.write(input_ocr_list)

    body = json.dumps(
        { 
            "anthropic_version":"bedrock-2023-05-31",
            "max_tokens": 4096,
            "temperature": 0.0,
            "system": system_prompt,
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": final_prompt.replace(
                                "<input_ocr_list/>",
                                """<input_ocr_list>"""+input_ocr_list+"""</input_ocr_list>"""
                            )
                        }
                    ],
                }
            ]
        }    
    )

    response = bedrock_rt.invoke_model(body=body, modelId=modelId)
    
    # print("--------------------------")
    # print(response)
    # print("--------------------------")
    
    response_body = json.loads(response.get('body').read())
    output = response_body['content'][0]['text']
    
    # print(output)
    
    # pattern = r'<result>(.*?)</result>'
    # match = re.search(pattern, output, re.DOTALL)
    # json_text = match.group(1)
    
    # data = json.loads(json_str)
    # print(data)
    # JSON 텍스트에서 <result> 태그 제거
    # json_text = output.replace("<result>", "").replace("</result>", "")

    # print(output)
    # JSON 데이터로 파싱
    # data = json.loads(output)
    
    # print(data)
    # end_time = time.time()
    # 수행 시간 계산 및 출력
    # total_time = execution_time + end_time - start_time
    # total_time_text = f"수행 시간: {total_time:.6f} 초"
    
    # JSON 데이터 출력
    # st.write(total_time_text)
    # st.json(data)
    return output

result_json = get_final_result(input_ocr_list, system_prompt, final_prompt)
print(result_json)


NameError: name 'input_ocr_list' is not defined

In [28]:
### 2-8 Json 결과 s3에 저장

# s3_bucket = "gsshop-video-analysis-com-761482380245"
# prefix = "images/prd_01/vrid_00001/"
split_result = prefix.split("/")
result_path_key = f"results/{split_result[2]}.json"

s3.put_object(
    Bucket=s3_bucket,
    Key=result_path_key,
    Body=result_json,
    ContentType='application/json'
)


{'ResponseMetadata': {'RequestId': 'XT7SM43RH4NG6YDT',
  'HostId': 'd37tkSlmLSiZKMNtm6X5drnaFiQRM0Iy4azdGSZAyX5Nh1MQFFpW8vuLYQo+QFUd5vpkWKoKOHY=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'd37tkSlmLSiZKMNtm6X5drnaFiQRM0Iy4azdGSZAyX5Nh1MQFFpW8vuLYQo+QFUd5vpkWKoKOHY=',
   'x-amz-request-id': 'XT7SM43RH4NG6YDT',
   'date': 'Fri, 23 Aug 2024 13:27:07 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"7c7ef954abb77b2bf59e2af64a90b950"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"7c7ef954abb77b2bf59e2af64a90b950"',
 'ServerSideEncryption': 'AES256'}

In [53]:
### 2-10. Upstage OCR Endpoint 삭제

sagemaker_session.delete_endpoint(
    endpoint_name=endpoint_name
)

print(f"Deleted endpoint: '{endpoint_name}'")

## [참고] 초기화

In [24]:
import s3_functions

bucket = "gsshop-video-analysis-761482380245-ap-northeast-2"
 
## 참고 폴더 원복
### 2-9. 분석 후 폴더 이동
def move_s3_folder(bucket, source_folder, destination_folder):

    # 소스 폴더의 모든 객체 나열
    response = s3.list_objects_v2(Bucket=bucket, Prefix=source_folder)

    # 'Contents' 키가 응답에 있는지 확인
    if 'Contents' not in response:
        print(f"No objects found in {source_folder}")
        return

    for obj in response['Contents']:
        old_key = obj['Key']
        new_key = old_key.replace(source_folder, destination_folder, 1)

        # 객체 복사
        s3.copy_object(
            Bucket=bucket,
            CopySource={'Bucket': bucket, 'Key': old_key},
            Key=new_key
        )

        # 원본 객체 삭제
        s3.delete_object(Bucket=bucket, Key=old_key)

        print(f"Moved {old_key} to {new_key}")

staging_prefix = "images/done/"
success_files = s3_functions.find_success_files(bucket, staging_prefix)

for key in success_files:
    # video_analyze(key)
    prefix = key.replace("_SUCCESS","")
    source_folder = prefix
    destination_folder = source_folder.replace("done","staging")
    s3_functions.move_s3_folder(bucket, source_folder, destination_folder)



Moved images/done/prd_01/vrid_00001/ to images/staging/prd_01/vrid_00001/
Moved images/done/prd_01/vrid_00001/_SUCCESS to images/staging/prd_01/vrid_00001/_SUCCESS
Moved images/done/prd_01/vrid_00001/frame_0000.jpg to images/staging/prd_01/vrid_00001/frame_0000.jpg
Moved images/done/prd_01/vrid_00001/frame_0001.jpg to images/staging/prd_01/vrid_00001/frame_0001.jpg
Moved images/done/prd_01/vrid_00001/frame_0002.jpg to images/staging/prd_01/vrid_00001/frame_0002.jpg
Moved images/done/prd_01/vrid_00001/frame_0003.jpg to images/staging/prd_01/vrid_00001/frame_0003.jpg
Moved images/done/prd_01/vrid_00001/frame_0004.jpg to images/staging/prd_01/vrid_00001/frame_0004.jpg
Moved images/done/prd_01/vrid_00001/frame_0005.jpg to images/staging/prd_01/vrid_00001/frame_0005.jpg
Moved images/done/prd_01/vrid_00001/frame_0006.jpg to images/staging/prd_01/vrid_00001/frame_0006.jpg
Moved images/done/prd_01/vrid_00001/frame_0007.jpg to images/staging/prd_01/vrid_00001/frame_0007.jpg
Moved images/done/pr

In [25]:
result_prefix = "results/"

response = s3.list_objects_v2(Bucket=bucket, Prefix=result_prefix)

success_files = []
if 'Contents' in response:
    for obj in response['Contents']:
        key = obj['Key']
        print(key)
        s3.delete_object(Bucket=bucket, Key=key)



results/vrid_00001.json
results/vrid_00002.json
results/vrid_00003.json
